In [20]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import string
color = sns.color_palette()
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import numpy as np

In [21]:
#Import data
attendance_valuation_elo_df = pd.read_csv("nba_2017_att_val_elo.csv")
salary_df = pd.read_csv("nba_2017_salary.csv")
pie_df = pd.read_csv("nba_2017_pie.csv")
plus_minus_df = pd.read_csv("nba_2017_real_plus_minus.csv")
br_stats_df = pd.read_csv("nba_2017_br.csv")

#Convert player names into the same format
plus_minus_df.rename(columns={"NAME":"PLAYER", "WINS": "WINS_RPM"}, inplace=True)
players = []
for player in plus_minus_df["PLAYER"]:
    plyr, _ = player.split(",")
    players.append(plyr)
plus_minus_df.drop(["PLAYER"], inplace=True, axis=1)
plus_minus_df["PLAYER"] = players
plus_minus_df.head()

,TEAM,GP,MPG,ORPM,DRPM,RPM,WINS_RPM,PLAYER
0,CLE,74,37.8,6.49,1.93,8.42,20.43,LeBron James
1,GS,79,33.4,7.27,0.14,7.41,18.80,Stephen Curry
2,CHI,76,37.0,4.82,1.80,6.62,17.35,Jimmy Butler
3,OKC,81,34.6,6.74,-0.47,6.27,17.34,Russell Westbrook
4,GS,76,32.5,1.55,5.59,7.14,16.84,Draymond Green


In [22]:
nba_players_df = br_stats_df.copy()
nba_players_df.rename(columns={'Player': 'PLAYER','Pos':'POSITION', 'Tm': "TEAM", 'Age': 'AGE', "PS/G": "POINTS"}, inplace=True)
nba_players_df.drop(["G", "GS", "TEAM"], inplace=True, axis=1)
nba_players_df = nba_players_df.merge(plus_minus_df, how="inner", on="PLAYER")
pie_df_subset = pie_df[["PLAYER", "PIE", "PACE", "W"]].copy()
nba_players_df = nba_players_df.merge(pie_df_subset, how="inner", on="PLAYER")
salary_df.rename(columns={'NAME': 'PLAYER'}, inplace=True)
salary_df["SALARY_MILLIONS"] = round(salary_df["SALARY"]/1000000, 2)
salary_df.drop(["POSITION","TEAM", "SALARY"], inplace=True, axis=1)
nba_players_with_salary_df = nba_players_df.merge(salary_df)
nba_players_with_salary_df.shape

(342, 38)

In [35]:
nba_players_with_salary_df.dtypes

Rk                   int64
PLAYER              object
POSITION            object
AGE                  int64
MP                 float64
FG                 float64
FGA                float64
FG%                float64
3P                 float64
3PA                float64
3P%                float64
2P                 float64
2PA                float64
2P%                float64
eFG%               float64
FT                 float64
FTA                float64
FT%                float64
ORB                float64
DRB                float64
TRB                float64
AST                float64
STL                float64
BLK                float64
TOV                float64
PF                 float64
POINTS             float64
TEAM                object
GP                   int64
MPG                float64
ORPM               float64
DRPM               float64
RPM                float64
WINS_RPM           float64
PIE                float64
PACE               float64
W                    int64
S

In [33]:
nba_players_with_salary_df.sort_values("2PA",ascending=False).loc[:,["2PA","PLAYER"]]

,2PA,PLAYER
3,18.6,Anthony Davis
0,16.8,Russell Westbrook
14,15.6,Andrew Wiggins
17,14.9,John Wall
4,14.8,DeMarcus Cousins
11,14.7,Karl-Anthony Towns
39,14.3,Derrick Rose
23,14.1,Blake Griffin
41,13.8,LaMarcus Aldridge
9,13.6,Kyrie Irving


In [125]:
nba_players_with_salary_df.to_csv("withrookies.csv")

In [128]:
nba_players_with_salary_df.groupby("AGE").SALARY_MILLIONS.max()

AGE
19     5.28
20     5.33
21     6.01
22     4.83
23    22.12
24    18.50
25    17.64
26    24.33
27    26.50
28    26.50
29    26.54
30    26.54
31    23.18
32    30.96
33    11.13
34    14.45
35    23.20
36    15.50
38    25.00
39    14.00
40     4.26
Name: SALARY_MILLIONS, dtype: float64

In [141]:
nba_players_with_salary_df[nba_players_with_salary_df["AGE"]==32]

,Rk,PLAYER,POSITION,AGE,MP,FG,FGA,FG%,3P,3PA,...,GP,MPG,ORPM,DRPM,RPM,WINS_RPM,PIE,PACE,W,SALARY_MILLIONS
6,8,LeBron James,SF,32,37.8,9.9,18.2,0.548,1.7,4.6,...,74,37.8,6.49,1.93,8.42,20.43,18.3,98.38,51,30.96
19,22,Carmelo Anthony,SF,32,34.3,8.1,18.8,0.433,2.0,5.7,...,74,34.3,1.87,-1.75,0.12,5.26,11.8,98.48,29,24.56
30,34,Marc Gasol,C,32,34.2,7.2,15.7,0.459,1.4,3.6,...,74,34.2,1.86,0.72,2.58,8.69,15.1,94.43,40,21.17
105,123,J.J. Barea,PG,32,22.0,4.1,9.8,0.414,1.5,4.2,...,35,22.0,0.15,-1.57,-1.42,0.80,13.4,95.14,11,4.10
108,126,Marcin Gortat,C,32,31.2,4.8,8.2,0.579,0.0,0.0,...,82,31.2,-1.25,1.18,-0.07,5.01,10.9,99.85,49,12.00
166,201,Al Jefferson,C,32,14.1,3.6,7.1,0.499,0.0,0.0,...,66,14.1,-1.00,-0.45,-1.45,0.98,13.3,96.89,33,10.31
185,227,Thabo Sefolosha,SF,32,25.7,2.8,6.4,0.441,0.7,1.9,...,62,25.7,-1.93,2.27,0.34,3.55,8.6,99.53,32,3.85
256,328,Aaron Brooks,PG,32,13.8,1.9,4.6,0.403,0.7,2.0,...,65,13.8,-1.81,-1.47,-3.28,-0.09,6.2,96.55,36,2.70
263,337,Tiago Splitter,C,32,9.5,1.8,3.9,0.452,0.3,0.8,...,8,9.5,-1.97,0.69,-1.28,0.09,10.6,99.11,1,8.55
300,405,Andrew Bogut,C,32,21.6,1.4,3.0,0.469,0.0,0.0,...,27,21.6,-4.01,4.34,0.33,1.20,9.2,93.14,8,0.24


#  Remove Rookies

In [91]:
# Get list of Rookie names to remove
rookie_list = pd.read_csv("nba_2017_rookies.csv",usecols=["PLAYER"])
def fix_string(strg):
    newstrg = ""
    acc = """ '",{}[].`;:  """
    for x in strg:
        if x in string.ascii_letters or x in string.digits or x in acc:
            newstrg += x
    return newstrg

rookie_list = rookie_list.PLAYER.apply(lambda x: fix_string(x))
remove_rookie_lst = ""
for i in rookie_list:
    if remove_rookie_lst =="":
        remove_rookie_lst+= i
    else:
        remove_rookie_lst = remove_rookie_lst +"|" + i

In [92]:
# Remove Rookie names: age <= 19 and based on rookie name
no_rookie = nba_players_with_salary_df[nba_players_with_salary_df.AGE>19]
no_rookie = no_rookie[no_rookie.PLAYER.str.contains(remove_rookie_lst,regex=True)==False]

In [93]:
# Mean salary for duplicated names
def deal_with_dup(no_rookie):
    no_rookie.drop_duplicates(inplace=True)
    dup = no_rookie[no_rookie.duplicated("PLAYER",keep=False)]
    nodup = no_rookie[no_rookie.duplicated("PLAYER",keep=False)!=True]
    lst = dup.columns.tolist()
    lst.remove("SALARY_MILLIONS")
    dup = dup.groupby(by = lst)["SALARY_MILLIONS"].mean().reset_index()
    return pd.concat([nodup,dup])
df = deal_with_dup(no_rookie)

In [123]:
# Impute NaN with Mean
def fix_na(df):
    df1 = df.copy()
    lst = df1.columns[df.isna().sum()>0]
    print(lst)
    for i in lst:
        no_na = df1[df1[i].notna()]
        na = df1[df1[i].isna()]
        na.fillna(np.mean(no_na[i]),inplace=True) 
        df1 = pd.concat([no_na, na])
    return df1
df1 = fix_na(df)
df1.to_csv("df_cleaned.csv")

Index(['3P%', 'FT%'], dtype='object')


In [ ]:
# Should we deal with injured players?

In [113]:
# np.mean(df[df['3P%'].notna()]['3P%'])\
df1.isna().sum().sum()

0

In [124]:
df1.shape

(297, 38)

NameError: name 'df1' is not defined